# 1. StrOutputParser()

In [4]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv

from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate

load_dotenv()

model = ChatGroq(model="gemma2-9b-it")

In [5]:
# 1st prompt -> detailed report
template1 = PromptTemplate(
    template='Write a detailed report on {topic}',
    input_variables=['topic']
)

# 2nd prompt -> summary
template2 = PromptTemplate(
    template='Write a 5 line summary on the following text. /n {text}',
    input_variables=['text']
)

In [6]:
prompt1= template1.invoke({'topic': 'blackhole'})
prompt1

StringPromptValue(text='Write a detailed report on blackhole')

In [8]:
result= model.invoke(prompt1)
result

AIMessage(content="## A Journey into the Abyss: A Detailed Report on Black Holes\n\nBlack holes, celestial objects of immense density and gravitational pull, have captured the imaginations of scientists and the public alike. These enigmatic entities, born from the death throes of massive stars, warp the fabric of spacetime, bending light and trapping anything that ventures too close. This report delves into the fascinating world of black holes, exploring their formation, properties, and the profound implications they have for our understanding of the universe.\n\n**1. Genesis of a Monster: Formation of Black Holes**\n\nBlack holes are not cosmic vacuum cleaners, randomly swallowing everything in their path. They are formed under specific, extreme conditions:\n\n* **Stellar Collapse:** When a star significantly more massive than our Sun exhausts its nuclear fuel, its core collapses under its own gravity. If the core's mass exceeds a critical threshold (approximately three times the mass

In [9]:
prompt2 = template2.invoke({'text': result.content})
result = model.invoke(prompt2)
result.content

'Black holes are incredibly dense objects formed from the collapse of massive stars or the merger of neutron stars. They are characterized by their mass, spin, and charge, and possess an event horizon beyond which escape is impossible.  Black holes are detected indirectly through their gravitational effects on surrounding matter, such as orbital motion, X-ray emissions, and gravitational waves. They play a crucial role in galaxy evolution, regulating star formation and influencing the dynamics of their host galaxies.  Ongoing research seeks to unravel mysteries surrounding black holes, including the information paradox and their behavior at the quantum level.  \n\n\n'

## Same code with StrOutputParser

In [10]:
parser = StrOutputParser()

chain = template1 | model | parser | template2 | model | parser

In [11]:
result= chain.invoke({'topic': 'blackhole'})
result

'Black holes are regions of spacetime with immense gravity where nothing, not even light, can escape. They form from the collapse of massive stars and are classified by mass as stellar-mass, supermassive, or intermediate-mass.  \n\nBlack holes are detected indirectly through their gravitational effects on surrounding matter, X-ray emissions from accretion disks, and gravitational waves produced by mergers. These cosmic objects play a crucial role in galaxy evolution and serve as laboratories for testing theories of gravity.  \n\n\nFuture research aims to characterize black hole populations, study accretion processes, and explore the enigmatic nature of singularities. \n'

In [14]:
# Full code

from langchain_groq import ChatGroq
from dotenv import load_dotenv

from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate

load_dotenv()

model = ChatGroq(model="gemma2-9b-it")

# 1st prompt -> detailed report
template1 = PromptTemplate(
    template='Write a detailed report on {topic}',
    input_variables=['topic']
)

# 2nd prompt -> summary
template2 = PromptTemplate(
    template='Write a 5 line summary on the following text. /n {text}',
    input_variables=['text']
)


parser= StrOutputParser()

chain = template1 | model | parser | template2 | model | parser

chain.invoke({'topic':'potato'})

'The potato, native to the Andes, has become a global staple crop.  Domesticated over 7,000 years ago, it spread to Europe and gained widespread acceptance, particularly in Ireland.  Rich in carbohydrates, vitamin C, and potassium, potatoes are versatile and used in countless dishes worldwide.  The global potato industry is significant, contributing to food security and generating billions in revenue.  Facing challenges like climate change, research focuses on developing resilient potato varieties to ensure future production. \n\n\n'

# 2. JsonOutputParser()

In [2]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv

from langchain_core.output_parsers import JsonOutputParser
from langchain.prompts import PromptTemplate

load_dotenv()

model = ChatGroq(model="gemma2-9b-it")

In [3]:
parser = JsonOutputParser()

In [4]:
template = PromptTemplate(
    template='Give me 5 facts about {topic} \n {format_instruction}',
    input_variables=['topic'],
    partial_variables={'format_instruction': parser.get_format_instructions()}
)

In [5]:
parser.get_format_instructions()

'Return a JSON object.'

In [7]:
chain = template | model | parser

result = chain.invoke({"topic": "black hole"})

result

{'facts': ['Black holes are regions in spacetime where gravity is so strong that nothing, not even light, can escape.',
  'They are formed when massive stars collapse at the end of their life cycle.',
  'Black holes are characterized by their event horizon, a boundary beyond which escape is impossible.',
  'Although we cannot see black holes directly, their presence can be inferred by observing their effects on surrounding matter.',
  'The most common type of black hole is a stellar-mass black hole, which is a few times more massive than our Sun.']}

# 3. StructuredOutputParser()

In [30]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv

from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import PromptTemplate

load_dotenv()

model = ChatGroq(model="gemma2-9b-it")

In [31]:
schema = [
    ResponseSchema(name='fact_1', description='Fact 1 about the topic'),
    ResponseSchema(name='fact_2', description='Fact 2 about the topic'),
    ResponseSchema(name='fact_3', description='Fact 3 about the topic')

]

In [32]:
parser = StructuredOutputParser.from_response_schemas(schema)

In [33]:
template = PromptTemplate(
    template= "Give 3 facts about {topic} \n {format_instruction}",
    input_variables= ['topic'],
    partial_variables = {'format_instruction': parser.get_format_instructions()}
)

In [ ]:
chain = template | model | parser

result = chain.invoke({'topic':'langchain'})
result

{'fact_1': 'LangChain is an open-source framework designed to simplify the development of applications powered by large language models (LLMs).',
 'fact_2': 'It provides tools for tasks like prompt engineering, chain creation, memory management, and integration with various LLMs and data sources.',
 'fact_3': 'LangChain enables developers to build more sophisticated and customizable AI applications by chaining together different LLM components and external services.'}

# 3. PydanticOutputParser()

In [20]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv

from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

from langchain.prompts import PromptTemplate

load_dotenv()

model = ChatGroq(model="gemma2-9b-it")

In [21]:
class Person(BaseModel):
    name: str = Field(description="Name of the person")
    age: int = Field(gt=18, description="age of the person")
    city : str =  Field(description='Name of the city the person belongs to')

In [ ]:
parser = PydanticOutputParser(pydantic_object=Person)

In [26]:
template = PromptTemplate(
    template='Generate the name, age and city of a fictional {place} person \n {format_instruction}',
    input_variables=['place'],
    partial_variables={'format_instruction': parser.get_format_instructions()}
)

In [27]:
parser.get_format_instructions()

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"name": {"description": "Name of the person", "title": "Name", "type": "string"}, "age": {"description": "age of the person", "exclusiveMinimum": 18, "title": "Age", "type": "integer"}, "city": {"description": "Name of the city the person belongs to", "title": "City", "type": "string"}}, "required": ["name", "age", "city"]}\n```'

In [28]:
chain = template | model | parser

final_result = chain.invoke({'place':'indian'})

print(final_result)

name='Anika Sharma' age=25 city='Mumbai'
